In [ ]:
import numpy as np
import torch
from scipy.io import loadmat
from cust_plot import plot
from ml import BrainDataset, reward

# Evaluate train/test results

In [ ]:
# Plot training data
data = torch.load('results/demo.pt', map_location=torch.device('cpu'))
plt_data = {k: data[k] for k in ('rewards','success','mu','sig')}
num_fns = 4
plot(plt_data, num_fns)

In [ ]:
# Load test data
res = 68
subj = [i + 1 for i in data['test_idx']]
sc = loadmat(f'data/subjfiles_SC{res}.mat')
fc = loadmat(f'data/subjfiles_FC{res}.mat')
sc = np.array([sc[f's{str(z+1).zfill(3)}'] for z in subj])
fc = np.array([fc[f's{str(z+1).zfill(3)}'] for z in subj])
euc_dist = loadmat('data/euc_dist.mat')[f'eu{res}']
hubs = np.loadtxt(f'data/hubs_{res}.txt', dtype=np.int, delimiter=',')
test_data = BrainDataset(sc, fc, euc_dist, hubs)

In [ ]:
# Eval test data
for i in range(len(test_data)):
    adj, sp, pp = test_data[i]
    pp.fn_weights = np.array(plt_data['mu'])[:,-1].tolist()
    mask = np.where(sp > 0)
    pred = pp.retrieve_all_paths()[mask]
    print(f'---- Test: {i+1} ----')
    print('Reward (global navigation efficiency ratio):', reward(pred, sp[mask]))
    print('Success:',1 - (pred == -1).sum() / len(pred))